## Домашнее задание

1. Необходимо подготовить датасет https://www.kaggle.com/olekslu/makeup-lips-segmentation-28k-samples для обучения модели на сегментацию губ
2. Обучить модель на выбор из [segmentation_models_pytorch](https://segmentation-modelspytorch.readthedocs.io/en/latest/index.html)

In [1]:
!pip install segmentation-models-pytorch

import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import os
from segmentation_models_pytorch import Unet
import torch.nn.functional as F
from torch import optim

In [2]:
class LipDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform

        self.image_files = os.listdir(images_dir)
        self.mask_files = os.listdir(masks_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        mask_path = os.path.join(self.masks_dir, self.mask_files[idx])

        image = Image.open(image_path)
        mask = Image.open(mask_path).convert('L')  # изменение режима изображения на 'L'

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

In [3]:
# путь к папке с изображениями губ
images_dir = "/Users/annvorosh/Documents/GB/PyTorch/set-lipstick-original/720p"

# путь к папке с масками губ
masks_dir = "/Users/annvorosh/Documents/GB/PyTorch/set-lipstick-original/mask"

In [4]:
# В датасете обнаружено несоответствие количества файлов изображений и их масок, 
# поэтому провели предобработку данных и удалили лишние файлы

import re

# Получаем список имен файлов
image_files = sorted(os.listdir(images_dir))
mask_files = sorted(os.listdir(masks_dir))

# Извлекаем числа из имен файлов
image_numbers = [int(re.findall('\d+', f)[0]) for f in image_files]
mask_numbers = [int(re.findall('\d+', f)[0]) for f in mask_files]

# Сравниваем числа в именах файлов
if set(image_numbers) != set(mask_numbers):
    print('Несоответствие имен файлов в папках images_dir и masks_dir:')
    print('Отсутствующие номера в images_dir:', set(mask_numbers) - set(image_numbers))
    print('Отсутствующие номера в masks_dir:', set(image_numbers) - set(mask_numbers))
    
# Находим отсутствующие номера в masks_dir
missing_numbers = set(image_numbers) - set(mask_numbers)

# Находим имена файлов, которые нужно удалить из images_dir
files_to_delete = [f for f in image_files if int(f.split(".")[0][5:]) in missing_numbers]
print('Лишние файлы:',files_to_delete)

# # Удаляем файлы
# for file in files_to_delete:
#     os.remove(os.path.join(images_dir, file))

# Переложим лишние файлы в другую папку
import shutil

destination_dir = "/Users/annvorosh/Documents/GB/PyTorch/set-lipstick-original/invalid_images"

# Переносим невалидные файлы
for file in files_to_delete:
    shutil.move(os.path.join(images_dir, file), os.path.join(destination_dir, file))

    
# Получаем список имен файлов
image_files = sorted(os.listdir(images_dir))
mask_files = sorted(os.listdir(masks_dir))

Несоответствие имен файлов в папках images_dir и masks_dir:
Отсутствующие номера в images_dir: set()
Отсутствующие номера в masks_dir: {10752, 10754, 10756, 10759, 10761, 10763, 10764, 10766, 10767, 10768, 10769, 10770, 10773, 10774, 10776, 10780, 10781, 10783, 10786, 10788, 10793, 10795, 10799, 10800, 10801, 10802, 10803, 10804, 10805, 10806, 10807, 10810, 10811, 10813, 10716, 10717, 10718, 10719, 10721, 10722, 10724, 10730, 10732, 10733, 10735, 10736, 10737, 10740, 10742, 10743, 10744, 10745, 10747, 10751}
['image00010716.jpg', 'image00010717.jpg', 'image00010718.jpg', 'image00010719.jpg', 'image00010721.jpg', 'image00010722.jpg', 'image00010724.jpg', 'image00010730.jpg', 'image00010732.jpg', 'image00010733.jpg', 'image00010735.jpg', 'image00010736.jpg', 'image00010737.jpg', 'image00010740.jpg', 'image00010742.jpg', 'image00010743.jpg', 'image00010744.jpg', 'image00010745.jpg', 'image00010747.jpg', 'image00010751.jpg', 'image00010752.jpg', 'image00010754.jpg', 'image00010756.jpg', 'i

In [5]:
# размер изображений и масок
image_size = (256, 256)

# загрузка датасета и разделение на тренировочную и валидационную выборки
dataset = LipDataset(images_dir, masks_dir, transform=transforms.Compose([transforms.Resize(image_size),
                                                                           transforms.ToTensor()]))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# загрузка тренировочной и валидационной выборки в DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [6]:
# создание экземпляра модели UNet и определение функции потерь
model = Unet('resnet34', encoder_weights='imagenet', classes=1, activation=None)
criterion = F.binary_cross_entropy_with_logits

# определение оптимизатора и scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

In [ ]:
# обучение модели
for epoch in range(2):
    train_loss = 0
    val_loss = 0

    # тренировочный этап
    model.train()
    for images, masks in train_loader:
        optimizer.zero_grad()

        output_masks = model(images)
        loss = criterion(output_masks, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # валидационный этап
    model.eval()
    with torch.no_grad():
        for images, masks in val_loader:
            output_masks = model(images)
            loss = criterion(output_masks, masks)
            val_loss += loss.item() * images.size(0)

    # выводим средний loss на каждой эпохе
    train_loss /= len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    print(f"Epoch {epoch+1}. Train loss: {train_loss:.4f}. Val loss: {val_loss:.4f}")

    # обновление scheduler
    scheduler.step(val_loss)

Epoch 1. Train loss: 0.0188. Val loss: 0.0157


In [ ]:
# сохранение обученной модели
torch.save(model.state_dict(), "lip_segmentation_model.pth")